### Составьте минимум 5 гипотез, на основе проведенного анализа, не ограничивайте себя, дайте волю фантазии, гипотезы могут быть самые невероятные.

Оформите результат в виде таблицы
- Следуйте структуре. Не критикуйте себя на этом этапе.
- Какие гипотезы можно сразу отмести и почему?

In [85]:
import pandas as pd

In [86]:
# Загружаем датасет по сессиям пользователей мобильного приложения Gett.
df = pd.read_csv('new_dataframe.csv')
df.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [87]:
# Изучаем показатели этих данных
df.describe(include='all')

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
count,101500.000000,101500.000000,101500.000000,101500,101500,91431,101500.0,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,91431.000000,101500.000000,101500
unique,NaN,NaN,NaN,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,Android,economy,no surge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,low
freq,NaN,NaN,NaN,56696,47219,59788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50436
mean,50749.500000,549874.802207,11.481429,NaN,NaN,NaN,1.0,0.900798,0.731389,0.620365,0.111025,0.571675,5.371152,25.906384,NaN
std,29300.670499,258600.104176,6.917522,NaN,NaN,NaN,0.0,0.298934,0.443239,0.485299,0.314164,0.494838,4.118531,7.838328,NaN
min,0.000000,100093.000000,0.000000,NaN,NaN,NaN,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010946,18.000000,NaN
25%,25374.750000,328223.000000,5.000000,NaN,NaN,NaN,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,2.438335,20.000000,NaN
50%,50749.500000,548827.000000,11.000000,NaN,NaN,NaN,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,4.279565,24.000000,NaN
75%,76124.250000,773051.000000,17.000000,NaN,NaN,NaN,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,7.129814,29.000000,NaN


Какие выводы можем сделать из этой информации?
- В среднем заходят в приложение в 11, совпадают тут медиана и среднее
- OS самый частый - Android, но отклонение не так значительно
- Класс самый частый - эконом из 3х уникальных, причем он занимает почти половину заказов
- Таблица содержит всех открывших приложение, более никаких данных тут не представлено

Дальнейшие некоторые вещи выражаю в процентах поскольку у нас принимаются лишь значения 1 и 0, а как следствие мы в среднем получаем процент
- В среднем к вбиванию адреса переходит (а как следствие к тому чтобы увидеть цену) 90%
- Заказ делает только 73% от всех вошедших в приложение
- Завершается поездка у 62% вошедших в приложение
- Отказывается от поездки 11% вошедших в приложение
- В центр заказывает 57% вошедших в приложение
- В среднем проезжают 5.3 км.
- Возраст ЦА - 26 лет, причем в межквартильном размахе находится именно аудитория между 20 и 29 годами. Очень мало людей в возрасте
- rfm как и surge = не совсем понятно что за метрики, скорее всего одна из них спрос, я бы уточнил, или попросил документацию, будь на работе

Давайте делать какие то более детальные расчеты, а также проверим целостность данных, чтобы предполагать уже какие то более конкретные вещи

Какие могут быть ошибки:
- User ID не уникален
- distance, user_cancelled,  указан там где не было вообще заказано машины

In [104]:
# Проверим UserID
# Клиент может поменять os, да и в целом что угодно, кроме резких скачков возраста 
max(df.groupby('user_id')['age'].std())

0.0

Судя по всему клиенты собраны за короткий промежуток времени, поскольку возраст не менялся, следовательно user_id привязан только к 1 конкретному аккаунту. вопрос что им могут пользоваться разные люди. Давайте сочтем так, что по настоящему уникальной будет считаться комбинация user_id и os. В таком случае нам станет проще доверять данным. Опять же ничего не мешает один акк использовать на телефонах одного типа, но такой разницей придется пренебречь



In [107]:
## Теперь проверим значение distance
df['distance'].count()/len(df) == df['price_seen'].mean()

True

Мы понимаем, что distance - это **не проеханное** расстояние, а лишь вбитое в приложение

In [108]:
# Проверим отличаются ли чем-то пользователи Android от пользователей Apple по среднему
df.groupby('os')[['hour', 'order_class', 'order_made','ride_completed', 'user_cancelled', 'city_center_order', 'distance', 'age']].mean(numeric_only=True)

,hour,order_made,ride_completed,user_cancelled,city_center_order,distance,age
os,,,,,,,
Android,11.499630,0.684369,0.554678,0.129692,0.563003,5.370664,25.952995
iOS,11.458397,0.790889,0.703486,0.087403,0.582649,5.371742,25.847402


Вот это интересно. Пользователи IOS на 15%  чаще заканчивают поездку. Остальные показатели практически идентичны

In [118]:
# Проверим отличаются ли чем-то пользователи Android от пользователей Apple по общему проеханному расстоянию и количеству поездок
df.loc[df['ride_completed'] == 1].groupby(['os'])[['distance','ride_completed']].sum(numeric_only=True)

,distance,ride_completed
os,,
Android,170727.004056,31448
iOS,170260.343183,31519


Притом расстояние они проезжают одинаковое и количество у нас в приложении поездок от одних и других одинаковое, но пользователей ios просто меньше

Отсюда берем гипотезу. Если перенаправить рекламный трафик на IOS пользователей, то наш заработок с этого клиента будет выше на 15%, чем если бы мы делали рекламу на Android, поскольку пользователь Ios заканчивает на 15% больше поездок

In [131]:
df_user_sum = df.groupby(['user_id', 'os'])[['app_opened', 'price_seen', 'order_made', 'ride_completed', 'user_cancelled', 'city_center_order', 'distance']].sum(numeric_only=True).copy()
df_user_sum
df_user_avg = df.groupby(['user_id', 'os'])[['hour', 'price_seen', 'order_made', 'ride_completed', 'user_cancelled', 'city_center_order', 'distance', 'age']].mean(numeric_only=True).copy()
df_user_avg.describe()

,hour,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age
count,42748.000000,42748.000000,42748.000000,42748.000000,42748.000000,42748.000000,41385.000000,42748.000000
mean,11.479384,0.902198,0.735875,0.626899,0.108976,0.571872,5.362035,27.171423
std,5.203100,0.222565,0.332524,0.366388,0.233936,0.372923,3.188825,8.853591
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074801,18.000000
25%,8.000000,1.000000,0.500000,0.400000,0.000000,0.333333,3.175825,20.000000
50%,11.500000,1.000000,1.000000,0.666667,0.000000,0.500000,4.748213,25.000000
75%,15.000000,1.000000,1.000000,1.000000,0.000000,1.000000,6.831635,31.000000
max,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.086622,69.000000


In [128]:
df_user_sum.describe()

,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance
count,42748.000000,42748.000000,42748.000000,42748.000000,42748.000000,42748.000000,42748.000000
mean,2.374380,2.138837,1.736596,1.472981,0.263615,1.357373,11.488018
std,1.311163,1.262892,1.164927,1.091438,0.511691,1.070638,9.117680
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,4.619709
50%,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,9.278830
75%,3.000000,3.000000,2.000000,2.000000,0.000000,2.000000,16.177889
max,11.000000,10.000000,10.000000,9.000000,4.000000,8.000000,83.324701


In [133]:
df.groupby('order_class').mean(numeric_only=True)

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age
order_class,,,,,,,,,,,
business,51081.323942,548314.764730,11.566027,1.0,0.897116,0.724092,0.640073,0.084019,0.568187,12.013842,25.917145
comfort,50831.878669,548849.747586,11.418481,1.0,0.899569,0.725502,0.629663,0.095839,0.575009,5.025267,25.933654
economy,50591.233804,551189.494144,11.516191,1.0,0.902836,0.738516,0.607023,0.131494,0.569580,3.997445,25.879244


In [134]:
df.groupby('order_class').sum(numeric_only=True)

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age
order_class,,,,,,,,,,,
business,614661571,6597871564,139174,12033,10795,8713,7702,1011,6837,129689.420250,311861
comfort,2147545210,23187804136,482408,42248,38005,30651,26602,4049,24293,190985.285881,1095645
economy,2388867469,26026616724,543783,47219,42631,34872,28663,6209,26895,170415.092055,1221992


Бизнес меньше отменяет поездки и выбирает более длинные пути

### Оцените гипотезы с точки зрения их привлекательности от 1 до 10.
Почему вы оценили их именно так? Свои рассуждения описывайте максимально подробно.

| N | Гипотеза | Источник | Почему не стоит проверять | Личный рейтинг | Почему такая оценка |
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |
| 1 | Если перенаправить рекламный трафик на IOS пользователей, то наш заработок с этого клиента будет выше на 15%, чем если бы мы делали рекламу на Android, поскольку пользователь Ios заканчивает на 15% больше поездок | Данные | Риск потратиться | 10 | Из за подкрепления данными, требует дальнейших исследований, но на данный момент выглядит правдоподобно |
| 2 | Если увеличить цену на 5% в 11 часов у всех пользователей, то наш заработок увеличится, поскольку данное время является пиковым  | Данные | Есть вероятность потери клиента и увеличения отказа после увиденной цены | 8 | Потому что таким образом мы можем отпугнуть клиентов в сторону конкурентов |
| 3 | Если увеличить стоимость за километр на 5%, то компания заработает дополнительные средства, поскольку бизнес и комфорт ездят дальше и на данный момент они заказывают так же часто, как и эконом после увиденной цены, а как следствие цену можно попробовать увеличить без повреждений | Данные | Есть вероятность потерять важных клиентов | 9 | Риск потери клиентов |
| 4 | Если ввести систему поощрений в эконом классе за то что каждый заказ заказанный подряд не был отменен мы сможем увеличить конверсию заказа в поездку на 5%, поскольку клиенты отменяют или когда передумали, или когда их не устроила цена/время прибытия и.т.д. Создав противовес мы заработаем до 5% больше из за того, что бизнес класс уже отменяет поездки на 5% реже | Данные | Стоимость и сложность создания системы мотивации | 10 | Сама идея не может отпугнуть клиента, только улучшить состояние. Вопрос только в стоимости разработки |
| 5 | Если сделать компанию по привлечению более взрослой аудитории на Ios, то мы сможем увеличить собственную клиентскую базу и заработать дополнительные средства | Данные, средний возраст пользователя | Есть вероятность что им это и не надо, рискованно, есть шанс потерять деньги | 5 | Потому что есть предположение, что люди в возрасте не так сильно заинтересованы в мобильности, хотя тут надо подумать, потому оценка не оч низкая |

### Приоритизируйте гипотезы по ICE
При расчете каждой составлющей ICE можно опираться на собственные представления о стоимости поездок, стоимость работ разработчиков и т.к. Главное понять смысл работы приоритезации

**Я возьму за цифру заработка компании в месяц сейчас 2.000.000р., представив, что это данные были полученные за месяц и километр стоил 40р.** Следовательно все показатели будут прыгать от этого

| N | Фича | Impact | Confidence | На чем основан | Effort | ICE | Ранг |
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |
| 1 | Перенапрение рекламного трафика | 200.000 | 60% | Согласно данным, предварительно, Ios пользователи приносят больше средств | 10.000 | 110.000 | 1 |
| 2 | Увеличение цены в пиковые часы | 100.000 | 40% | Данные часы используют абсолютно все клиенты как пиковые, следовательно незначительное увеличение стоимости не должно сказаться на юзерах. Данные + мнение | 40.000 | 0 | 4 |
| 3 | Увеличение цены за расстояние | 80.000 | 80% | Клиенты мало отменяют поездки после увиденной цены и, как следствие, могут заплатить столько и можно попробовать толкнуть эту ветку пока не увидим существенный спад. Данные | 20.000 | 44.000 | 2 |
| 4 | Система поощрений в эконом-классе за сохранение заказа | 40.000 | 90% | Согласно данным у эконома большое количество отказов после увиденной цены, снизив это мы можем повысить заработок с этого | 80.000 | -44.000 | 5 |
| 5 | Кампания для людей в возрасте | 200.000 | 30% | Согласно данным мы мало привлекаем людей в возрасте. Но они могут не интересоваться нашим продуктом вовсе. Достойно тестирвания, но рискованно | 40.000 | 20.000 | 3 |